# Agenda

1. What are decorators?
2. Decorating functions
3. Enclosing functions (and taking advantage of them with decorators)

Decorators in Python ≠ Decorators in design patterns

# Things to remember about functions in Python

1. When we use `def` to define a function, we are actually doing two things:
    - Creating a function object
    - Assigning that function object to a variable
2. Inside of a function, assigning to a variable creates a local variable
3. Functions are objects, which we can return (like any other object) from a function.

In [1]:
# let's say that we have two functions

def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(a())
print(b())

A!

B!



In [2]:
# our company has now decided, as a matter of policy, that anything we print
# in our programs must have dashed lines above and below it

lines = '-' * 10 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

----------
A!
----------

----------
B!
----------



In [3]:
# this violates the DRY rule -- don't repeat yourself

# how can we DRY up our code, and not repeat ourselves?

lines = '-' * 10 + '\n'

# this function will take another function as an argument, and put its output in lines
def with_lines(func):    
    return f'{lines}{func()}{lines}'


def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(with_lines(a))  # I'm not running a or b directly.. with_lines will run them for me
print(with_lines(b))

----------
A!
----------

----------
B!
----------



In [4]:
# the boss is unhappy that we changed the API
# we need to have lines... but we need to have them without changing how we call a and b

# we're going to solve this problem by having with_lines return a *function*, 
# rather than a string

lines = '-' * 10 + '\n'

def with_lines(func):    
    def wrapper():    # define an inner function
        return f'{lines}{func()}{lines}'
    return wrapper    # return the inner function

def a():
    return f'A!\n'
a_with_lines = with_lines(a)


def b():
    return f'B!\n'
b_with_lines = with_lines(b)

print(a_with_lines())
print(b_with_lines())

----------
A!
----------

----------
B!
----------



In [ ]:
# let's now make sure that we have the original API working

lines = '-' * 10 + '\n'

def with_lines(func): # func will hold onto our reference to our original function   
    def wrapper():    # define an inner function
        return f'{lines}{func()}{lines}'
    return wrapper    # return the inner function

def a():
    return f'A!\n'
a = with_lines(a)  # we're redefining a so that it's now the result of calling with_lines(a)


def b():
    return f'B!\n'
b = with_lines(b)

print(a_with_lines())
print(b_with_lines())